# Prepare All Catalogs

This notebook contains a list of functions that prepare the complementary catalogs for future work.

Make sure all of the functions add new columns to the catalogs instead of replace the values of the old was, to avoid replacing multiple times and making a mistake

It clears the catalogs of non-detections and flagged objects.

## Import libraries

In [8]:
# Astropy
from astropy.table import Table
from astropy.io import fits
from astropy.coordinates import SkyCoord
from astropy import units as u

import missingno
import numpy as np

## Define the paths to the main catalogs to use

In [9]:
# EMU catalogs
emu_catalog_path = '/data/mfonseca/survey_data/EMU_data/EMU_0102-32/EMU_0102-32_1comp.fits'
emu_patched_catalog_path = '/data/mfonseca/survey_data/EMU_data/EMU_0102-32/EMU_0102-32_1comp_withoutpatches.fits'

# DES catalogs
desdr2_catalog_path = '/data/mfonseca/survey_data/DES_data/DESDR2_in_EMU_0102-32_magautodered.fits'
desy6gold_catalog_path = '/data/mfonseca/survey_data/DES_data/DESY6GOLD_in_EMU_0102-32_magauto.fits'

# VIKING catalogs
vikingdr5_catalog_path = '/data/mfonseca/survey_data/VIKING_data/VIKINGDR5_in_EMU_0102-32_allapermag3.fits'

# CatWISE catalogs
catwise2020_catalog_path = '/data/mfonseca/survey_data/CATWISE_data/CATWISE2020_in_EMU_0102-32.fits'

## Convert VEGA magnitude to AB magnitude (VIKING DR5, CATWISE2020)

While the DES DR2 magnitudes are already in the AB system, the VIKING DR5 and CATWISE2020 magnitudes are in the Vega system and need to be converted.

In [10]:
def fix_ab_magnitude(file_path, ab_offset, mag_columns):
    '''
    Converts Vega magnitudes to AB magnitudes.
    Works for both VIKING DR5 and CATWISE 2020 catalogs.

    Args:
        file_path (str): Path to the FITS file.
        ab_offset (list): List of offsets for each magnitude column.
        mag_columns (list): List of magnitude column names to be corrected, same order as ab_offset.
    Returns:
        None: The function modifies the FITS file in place.
    '''
    catalog_table = Table.read(file_path)
    catalog_df = catalog_table.to_pandas()

    for idx, col in enumerate(mag_columns):
        new_col_name = col + '_ab'
        offset = ab_offset[idx]
        
        # Create the dered column, copying the original column first
        catalog_df[new_col_name] = catalog_df[col]

        # Apply the correction only where the value is NOT the bad value (only for VIKING DR5)
        mask = catalog_df[col] != -999999488.0
        catalog_df.loc[mask, new_col_name] = catalog_df.loc[mask, col] + offset

    # Save the updated DataFrame back to the original FITS file
    updated_catalog_table = Table.from_pandas(catalog_df)
    updated_catalog_table.write(file_path, overwrite=True)

For the ViKING DR5 we used the two magnitudes recomended in the webpage of VSA: AperMagNoAperCorr3 and AperMag3.

In [11]:
viking_AperMagNoAperCorr3_cols = ['zAperMagNoAperCorr3', 'yAperMagNoAperCorr3', 'jAperMagNoAperCorr3', 'hAperMagNoAperCorr3', 'ksAperMagNoAperCorr3']
viking_AperMag3_cols = ['zAperMag3', 'yAperMag3', 'jAperMag3', 'hAperMag3', 'ksAperMag3']


# http://casu.ast.cam.ac.uk/surveys-projects/vista/technical/filter-set
viking_ab_offset = [0.502, 0.600, 0.916, 1.366, 1.827]

fix_ab_magnitude(vikingdr5_catalog_path, viking_ab_offset, viking_AperMagNoAperCorr3_cols)
fix_ab_magnitude(vikingdr5_catalog_path, viking_ab_offset, viking_AperMag3_cols)

For the CATWISE2020 survey, given the big size of the aperture magnitudes, we use the PSF magnitudes.

In [12]:
# For CATWISE2020
catwise_psfmag_columns = ['w1mpro', 'w2mpro']

# https://wise2.ipac.caltech.edu/docs/release/allsky/expsup/sec4_4h.html
catwise_ab_offset = [2.699, 3.339]

fix_ab_magnitude(catwise2020_catalog_path, catwise_ab_offset, catwise_psfmag_columns)

## Correct for extinction (DESY6GOLD, VIKING DR5)

In [1]:
def correct_extinction(file_path, mag_extinction_dict, bad_value):
    '''
    Corrects the extinction in the magnitudes of the catalog using the total extinction.

    Args:
        file_path (str): Path to the FITS file.
        mag_extinction_dict (dict): Dictionary where
            key = name of the magnitude column,
            value = name of the extinction coefficient column.
    Returns:
        None: The function modifies the FITS file in place.
    '''

    catalog_table = Table.read(file_path)
    catalog_df = catalog_table.to_pandas()

    for mag_col, excoef_col in mag_extinction_dict.items():
        new_mag_col = mag_col + '_extcorr'

        # Initialize the new column with the original values
        catalog_df[new_mag_col] = catalog_df[mag_col]

        # Create a mask: where values are good (i.e., not the bad value)
        mask = catalog_df[mag_col] != bad_value

        # Apply extinction correction only on good values
        catalog_df.loc[mask, new_mag_col] = catalog_df.loc[mask, mag_col] - catalog_df.loc[mask, excoef_col]

    # Save the updated DataFrame back to the original FITS file
    updated_catalog_table = Table.from_pandas(catalog_df)
    updated_catalog_table.write(file_path, overwrite=True)

In [ ]:
# For DES Y6 GOLD
des_extinction_dict = {
    'mag_auto_g': 'a_fiducial_g',
    'mag_auto_r': 'a_fiducial_r',
    'mag_auto_i': 'a_fiducial_i',
    'mag_auto_z': 'a_fiducial_z',
    'mag_auto_y': 'a_fiducial_y',
}

correct_extinction(desy6gold_catalog_path, des_extinction_dict, 99)

In [ ]:
# For VIKING DR5
viking_extinction_dict = {
    'zAperMag3_ab': 'aZ',
    'yAperMag3_ab': 'aY',
    'jAperMag3_ab': 'aJ',
    'hAperMag3_ab': 'aH',
    'ksAperMag3_ab': 'aKs',
    'zAperMagNoAperCorr3_ab': 'aZ',
    'yAperMagNoAperCorr3_ab': 'aY',
    'jAperMagNoAperCorr3_ab': 'aJ',
    'hAperMagNoAperCorr3_ab': 'aH',
    'ksAperMagNoAperCorr3_ab': 'aKs',
}

correct_extinction(vikingdr5_catalog_path, viking_extinction_dict, -999999488.0)

## Calculate Colors (DES DR2, DES Y6 GOLD, VIKING DR5, CATWISE2020)

In [16]:
# For DES Y6GOLD
desy6gold_catalog_table = Table.read(desy6gold_catalog_path)
desy6gold_catalog_df = desy6gold_catalog_table.to_pandas()

desy6gold_mag_cols = ['mag_auto_g_extcorr', 'mag_auto_r_extcorr', 'mag_auto_i_extcorr', 'mag_auto_z_extcorr', 'mag_auto_y_extcorr']
desy6gold_catalog_df['g_r_extcorr'] = desy6gold_catalog_df['mag_auto_g_extcorr'] - desy6gold_catalog_df['mag_auto_r_extcorr']
desy6gold_catalog_df['r_i_extcorr'] = desy6gold_catalog_df['mag_auto_r_extcorr'] - desy6gold_catalog_df['mag_auto_i_extcorr']
desy6gold_catalog_df['i_z_extcorr'] = desy6gold_catalog_df['mag_auto_i_extcorr'] - desy6gold_catalog_df['mag_auto_z_extcorr']
desy6gold_catalog_df['z_y_extcorr'] = desy6gold_catalog_df['mag_auto_z_extcorr'] - desy6gold_catalog_df['mag_auto_y_extcorr']

updated_catalog_table = Table.from_pandas(desy6gold_catalog_df)
updated_catalog_table.write(desy6gold_catalog_path, overwrite=True)

In [36]:
# Define bad value
BAD_VALUE = -999999488.0

# Read the catalog
vikingdr5_catalog_table = Table.read(vikingdr5_catalog_path)
vikingdr5_catalog_df = vikingdr5_catalog_table.to_pandas()

# --- Helper function to compute colors safely ---
def compute_color(df, col1, col2, new_col_name):
    mask_good = (df[col1] != BAD_VALUE) & (df[col2] != BAD_VALUE)
    
    # Initialize the color column with bad values
    df[new_col_name] = BAD_VALUE

    # Where both magnitudes are good, compute the color
    df.loc[mask_good, new_col_name] = df.loc[mask_good, col1] - df.loc[mask_good, col2]

# --- AperMag3 (am3) colors ---
compute_color(vikingdr5_catalog_df, 'zAperMag3_ab_extcorr', 'yAperMag3_ab_extcorr', 'z_y_am3_extcorr')
compute_color(vikingdr5_catalog_df, 'yAperMag3_ab_extcorr', 'jAperMag3_ab_extcorr', 'y_j_am3_extcorr')
compute_color(vikingdr5_catalog_df, 'jAperMag3_ab_extcorr', 'hAperMag3_ab_extcorr', 'j_h_am3_extcorr')
compute_color(vikingdr5_catalog_df, 'hAperMag3_ab_extcorr', 'ksAperMag3_ab_extcorr', 'h_ks_am3_extcorr')

# --- AperMagNoAperCorr3 (am3nac) colors ---
compute_color(vikingdr5_catalog_df, 'zAperMagNoAperCorr3_ab_extcorr', 'yAperMagNoAperCorr3_ab_extcorr', 'z_y_am3nac_extcorr')
compute_color(vikingdr5_catalog_df, 'yAperMagNoAperCorr3_ab_extcorr', 'jAperMagNoAperCorr3_ab_extcorr', 'y_j_am3nac_extcorr')
compute_color(vikingdr5_catalog_df, 'jAperMagNoAperCorr3_ab_extcorr', 'hAperMagNoAperCorr3_ab_extcorr', 'j_h_am3nac_extcorr')
compute_color(vikingdr5_catalog_df, 'hAperMagNoAperCorr3_ab_extcorr', 'ksAperMagNoAperCorr3_ab_extcorr', 'h_ks_am3nac_extcorr')

# Save back to FITS
updated_catalog_table = Table.from_pandas(vikingdr5_catalog_df)
updated_catalog_table.write(vikingdr5_catalog_path, overwrite=True)

In [18]:
# For the CATWISE2020 catalog
catwise_catalog_table = Table.read(catwise2020_catalog_path)
catwise_catalog_df = catwise_catalog_table.to_pandas()

catwise_catalog_df['w1_w2_ab'] = catwise_catalog_df['w1mpro_ab'] - catwise_catalog_df['w2mpro_ab']

updated_catalog_table = Table.from_pandas(catwise_catalog_df)
updated_catalog_table.write(catwise2020_catalog_path, overwrite=True)

# Count non-detections (VIKING DR5, CATWISE2020)

In [19]:
catalog_table = Table.read(vikingdr5_catalog_path)
vikingdr5_catalog_df = catalog_table.to_pandas()

print(vikingdr5_catalog_df.columns)

Index(['sourceID', 'ra', 'dec', 'zAperMag3', 'zAperMagNoAperCorr3',
       'zAperMag3Err', 'yAperMag3', 'yAperMagNoAperCorr3', 'yAperMag3Err',
       'jAperMag3', 'jAperMagNoAperCorr3', 'jAperMag3Err', 'hAperMag3',
       'hAperMagNoAperCorr3', 'hAperMag3Err', 'ksAperMag3',
       'ksAperMagNoAperCorr3', 'ksAperMag3Err', 'eBV', 'aZ', 'aY', 'aJ', 'aH',
       'aKs', 'zppErrBits', 'yppErrBits', 'ksErrBits', 'jErrBits', 'hErrBits',
       'mergedClassStat', 'pGalaxy', 'pStar', 'pNoise', 'pSaturated',
       'zAperMagNoAperCorr3_ab', 'yAperMagNoAperCorr3_ab',
       'jAperMagNoAperCorr3_ab', 'hAperMagNoAperCorr3_ab',
       'ksAperMagNoAperCorr3_ab', 'zAperMag3_ab', 'yAperMag3_ab',
       'jAperMag3_ab', 'hAperMag3_ab', 'ksAperMag3_ab', 'zAperMag3_ab_extcorr',
       'yAperMag3_ab_extcorr', 'jAperMag3_ab_extcorr', 'hAperMag3_ab_extcorr',
       'ksAperMag3_ab_extcorr', 'zAperMagNoAperCorr3_ab_extcorr',
       'yAperMagNoAperCorr3_ab_extcorr', 'jAperMagNoAperCorr3_ab_extcorr',
       'hAper

In [35]:
print(f'the nan value is: {vikingdr5_catalog_df['zAperMag3'].tail(len(vikingdr5_catalog_df)).unique().min()}')

z_filter = vikingdr5_catalog_df[vikingdr5_catalog_df['zAperMag3']== -999999488.0]
y_filter = vikingdr5_catalog_df[vikingdr5_catalog_df['yAperMag3'] == -999999488.0]
j_filter = vikingdr5_catalog_df[vikingdr5_catalog_df['jAperMag3'] == -999999488.0]
h_filter = vikingdr5_catalog_df[vikingdr5_catalog_df['hAperMag3'] == -999999488.0]
ks_filter = vikingdr5_catalog_df[vikingdr5_catalog_df['ksAperMag3'] == -999999488.0]


print(f'{len(z_filter)} ({(len(z_filter)/len(vikingdr5_catalog_df))*100:.3f}%) objects have a nan value in the z filter')
print(f'{len(y_filter)} ({(len(y_filter)/len(vikingdr5_catalog_df))*100:.3f}%) objects have a nan value in the y filter')
print(f'{len(j_filter)} ({(len(j_filter)/len(vikingdr5_catalog_df))*100:.3f}%) objects have a nan value in the j filter')
print(f'{len(h_filter)} ({(len(h_filter)/len(vikingdr5_catalog_df))*100:.3f}%) objects have a nan value in the h filter')
print(f'{len(ks_filter)} ({(len(ks_filter)/len(vikingdr5_catalog_df))*100:.3f}%) objects have a nan value in the ks filter')

the nan value is: -999999488.0
93357 (9.753%) objects have a nan value in the z filter
97536 (10.190%) objects have a nan value in the y filter
5 (0.001%) objects have a nan value in the j filter
10 (0.001%) objects have a nan value in the h filter
5 (0.001%) objects have a nan value in the ks filter


In [21]:
catalog_table = Table.read(catwise2020_catalog_path)
catwise_catalog_df = catalog_table.to_pandas()

In [22]:
# This shows us that catwise doesn't have nan values

# missingno.matrix(catwise_catalog_df)

print(f'the nan value is: {catwise_catalog_df['w1mpro'].tail(len(catwise_catalog_df)).unique().min()}')

the nan value is: 5.181000232696533


# Remove EMU sources in empty VIKING spots

There are three rectangular patches in the VIKING area that don't have any objects, which is due to bad quality measurments.

In [23]:
emu_catalog_path = '/data/mfonseca/survey_data/EMU_data/EMU_0102-32/EMU_0102-32_1comp.fits'

In [24]:
emu_table = Table.read(emu_catalog_path)
emu_catalog_df = emu_table.to_pandas()

viking_table = Table.read(vikingdr5_catalog_path)
viking_catalog_df = viking_table.to_pandas()

emu_coords = SkyCoord(ra=emu_catalog_df['ra_deg_cont'].values * u.deg,
                      dec=emu_catalog_df['dec_deg_cont'].values * u.deg)

viking_coords = SkyCoord(ra=viking_catalog_df['ra'].values * u.deg,
                         dec=viking_catalog_df['dec'].values * u.deg)

# Perform many-to-many matching (find all EMU–VIKING pairs within 10 arcsec)
idx_viking, idx_emu, sep2d, _ = emu_coords.search_around_sky(viking_coords, 10 * u.arcsec)

# Get unique EMU indices that have at least one match
unique_matched_emu_indices = np.unique(idx_emu)

matched_emu_df = emu_catalog_df.iloc[unique_matched_emu_indices]

Table.from_pandas(matched_emu_df).write("/data/mfonseca/survey_data/EMU_data/EMU_0102-32/EMU_0102-32_1comp_filtered.fits", overwrite=True)

In [25]:
print(f'Number of original sources in EMU catalog: {len(emu_catalog_df)}')
print(f'Number of sources in EMU catalog after filtering: {len(matched_emu_df)}')

fraction = len(matched_emu_df) / len(emu_catalog_df)

print(f'Fraction of sources in EMU catalog after filtering: {fraction:.2%}')

Number of original sources in EMU catalog: 18736
Number of sources in EMU catalog after filtering: 11588
Fraction of sources in EMU catalog after filtering: 61.85%
